# Loading library

In [245]:
import pandas as pd
import numpy as np
import warnings
# Suppress RuntimeWarning: divide by zero encountered in divide
warnings.filterwarnings("ignore", category=RuntimeWarning)

# Functions

## Xử lý đầu vào

1. Về ràng buộc đẳng thức, bất đẳng thức --> chuyển về dạng chuẩn  
- nếu <=: giữ nguyên  
- nếu >=: nhân với -1  
- nếu =: nhân vế trái với -1  

2. Về ràng buộc dấu của biến 
- x>=0: giữ nguyên
- x<=0: thay x=-x, ghi nhớ vị trí biến để kết luận x=-x
- x không có ràng buộc: tạo 2 biến +x, -x mới, ghi nhớ vị trí biến để kết luận x = (+x) - (-x)

3. Về hướng của hàm mục tiêu  
- Nếu optimize_direction là max thì đưa hàm mục tiêu f(x) thành -f(-x)
- Nếu optimize_direction là min thì giữ nguyên

In [246]:
def processing_input(A,B):
    """ 
    # goal
    - chuyển 2 dataframe lưu ràng buộc đẳng thức/bất đẳng thức và ràng buộc về dấu của biến trong bài toán quy hoạch tuyến tính cho trước thành những ma trận có thể tính toán, xử lý được

    # input
    - A (dataframe): lưu ràng buộc đẳng thức/bất đẳng thức
    - B (dataframe): ràng buộc về dấu của biến 

    # output
    - optimize_direction: hướng của hàm mục tiêu
    - norm_arr (np.array): ma trận của bài toán sau khi chuyển về dạng chuẩn (P')
    - tab_arr (np.array): ma trận tương ứng với dạng bảng của phương pháp đơn hình của norm_arr
    - rsby (np.array): ma trận lưu trữ cách biểu diễn nghiệm của (P) theo (P')
    """

    # 1. Kiểm tra hướng của hàm mục tiêu
    # A.iloc[0] = A.iloc[0].fillna(0)
    A = A.fillna(0)
    optimize_direction = ''
    if A.iloc[0,-1] == 'max':
        optimize_direction = 'max'
    else:
        optimize_direction = 'min'
    A.iloc[0,-1] = 0
    # đưa cột cuối cùng của A về dạng numerical
    A[A.shape[1]-1] = pd.to_numeric(A[A.shape[1]-1])
    # nếu optimize_direction là max thì đưa hàm mục tiêu f(x) thành -f(-x)
    if optimize_direction == 'max':
        A.iloc[0,:] = -A.iloc[0,:]

    # 2. Xử lý ràng buộc về đẳng thức và bất đẳng thức
    # norm_arr là ma trận mới sau khi chuẩn hoá ma trận A. Dùng để làm biến đầu vào trong hàm xoay Bland
    A.iloc[0] = A.iloc[0].fillna(0)
    norm_arr = A.drop(columns=A.columns[A.shape[1] -1 -1]).to_numpy()
    nrow, ncol = norm_arr.shape
    for i in range(1,nrow):
        if A.iloc[i, -2] == ">=":
            norm_arr[i] = -norm_arr[i]
        elif A.iloc[i, -2] == "=":
            norm_arr[i, :-1] = -norm_arr[i, :-1]

    # 3. Xử lý ràng buộc về dấu của biến
    # rsby là hàm result by vars: dùng để kết luận biến cũ theo biến mới sau khi chạy xong thuật toán xoay Bland
    rsby = np.eye(B.shape[1])
    k = 0
    for j in range(B.shape[1]):
        if B.iloc[0, j] == "<=":
            norm_arr[:, k] = -norm_arr[:,k]
            rsby[:, k] = -rsby[:,k]
        elif B.iloc[0, j] == 0:
            new_col_norm_A = -norm_arr[:, k]
            norm_arr = np.insert(norm_arr, k+1, new_col_norm_A, axis = 1)
            new_col_rsby = -rsby[:, k]
            rsby = np.insert(rsby, k+1, new_col_rsby, axis = 1)
            k = k+1
        k = k+1

    # 4. đưa norm_arr về dạng bảng của phương pháp đơn hình tab_arr
    # Đưa về dạng ma trận để có thể giải bài toán x1|x2|...|xn|w1|w2|...|wn|bi 
    n_var = norm_arr.shape[1] - 1
    n_const = norm_arr.shape[0] -1
    arr = np.insert(np.zeros((1, n_const)), 1, np.eye(n_const), axis = 0)
    tab_arr = np.hstack((norm_arr[:, :n_var], arr, norm_arr[:, n_var:]))
    # cho tab_arr[1:,:-1] = -tab_arr[1:,:-1]
    tab_arr[1:,:-1] = -tab_arr[1:,:-1]
    
    return(norm_arr, tab_arr, rsby, optimize_direction)

## Bland Algorithm

In [247]:
def select_input_output_index(arr):
    """
    # goal 
    - tìm chỉ số dòng i, chỉ số cột j của biến cần xoay trong thuật toán Bland

    # input
    - arr (np.array): ma trận tương tự dạng bảng của phương pháp đơn hình

    # output
    - nếu i = None, tức là không có biến vào --> từ vựng tối ưu
    - nếu i != None, j = None, tức là có biến vào, không có biến ra --> bài toán không giới nội
    - nếu i != None, j != None, trả về chỉ số (i,j) cần tìm
    """
    # Find the index of the first negative element, or None if no negative elements exist.
    const_arr = arr[0,:-1]
    lst = const_arr[const_arr<0]
    ind_in = None
    ind_out = None
    if len(lst) != 0:
        ind_in = np.where(const_arr<0)[0][0]
    else:
        # Tu vung toi uu
        return('TVTU')
    
    # Find the index of the largest negative number 
    lst = arr[1:, -1]/[i if i<0 else np.nan for i in arr[1:, ind_in]]
    if len(lst[lst<=0]) == 0:
        ind_out = None
    else:
        ind_out = np.where(lst == max(lst[lst<=0]))[0][0] + 1

    # Bai toan Khong gioi noi
    if ind_in != None and ind_out == None:
        return('KGN') 
    
    # Tra ve vi tri cua bien can xoay
    return(ind_in, ind_out)
    

def find_eye_list(arr_col):
    """ 
    # goal
    - Tìm vị trí dòng của biến cơ sở ở từ vựng tối ưu. Dùng để kết luận nghiệm (xét ở trường hợp duy nhất nghiệm)
    - Ở từ vựng tối ưu, nếu x_i là biến cơ sở, thì cột x_i - arr_col là mảng chỉ có số 0 và -1. Và có duy nhất một số -1 
    
    # input
    - arr_col: ma trận cột của từ vựng tối ưu

    # output
    - vị trí dòng của biến cơ sở
    """
    # Finds the index of the unique element equal to -1 in a binary array.
    lst = arr_col.tolist()
    for element in lst:
        if element not in [0,-1]:
            return None
    if lst.count(-1) == 1:
        return lst.index(-1)

def bland_rotate(arr, i, j):
    """
    # goal
    - sử dụng thuật toán bland để xoay ma trận arr thành một ma trận mới arr_new

    # input
    - arr: mảng cần xoay
    - i: vị trí hàng của biến cần xoay
    - j: vị trí cột của biến cần xoay

    # output: return về mảng mới vừa được xoay
    """
    nrow, ncol = arr.shape
    arr_new = arr.copy()
    arr_new[i] = arr_new[i]/(-arr[i,j])
    for row in range(nrow):
        if row != i:
            for col in range(ncol):
                if col == j:
                    arr_new[row, col] = 0
                else:
                    arr_new[row, col] = arr[row, j] * arr_new[i, col] + arr[row, col]
    
    return arr_new

def bland_algoritm(tab_arr):
    """ 
    # goal: sử dụng thuật toán bland để giải quyết bài toán quy hoạch tuyến tính khi tồn tại ci<0 trên hàm mục tiêu và bi>=0 trên ràng buộc đẳng thức, bất đẳng thức sau khi chuẩn hoá

    # input
    - tab_arr (np.array): ma trận từ vựng được tạo thành bởi hàm mục tiêu; các ràng buộc đẳng thức, bất đẳng thức; các ràng buộc về dấu của biến đã được chuẩn hoá

    # output
    - df_result (list): một danh sách lưu các từ vựng/ các bước giải của thuật toán 2 pha
    - type_result: phân loại kết quả của từ vựng cuối cùng trong df_result: KGN, VN, TVTU(DDN, VSN) 
    """
    # tạo danh sách df_result để lưu lại các từ vựng tại mỗi lần xoay
    df_result = list()

    # thêm từ vựng ban đầu vào danh sách
    cur_arr = tab_arr
    df_result.append(cur_arr)

    # tạo biến type_result lưu các trường hợp xảy ra khi kết luận nghiệm: 
    # DDN: duy nhất nghiệm, KGN: không giới nội, VN: vô nghiệm, VSN: vô số nghiệm
    type_result = None

    # Xoay bland cho đến khi tìm được từ vựng tối ưu
    in_out_ind = select_input_output_index(cur_arr)
    while in_out_ind != 'TVTU':
        if in_out_ind != "KGN":
            j, i = in_out_ind
            new_arr = bland_rotate(cur_arr, i, j)
            df_result.append(new_arr)
        else:
            type_result = "KGN"
            break
        cur_arr = new_arr
        in_out_ind = select_input_output_index(cur_arr)
    
    type_result = select_input_output_index(df_result[-1])

    return(df_result, type_result)


def print_result(rsby, optimize_direction, df_result, type_result):
    """ 
    # goal
    - in kết quả của bài toán quy hoạch tuyến tính

    # input
    - rsby: ma trận biểu diễn của 1 biến
    - optimize_direction: biến lưu hướng tối ưu của hàm mục tiêu (max or min)
    - df_result: list lưu các bước giải của bài toán QHTT
    - type_result: phân loại kết quả của từ vựng cuối cùng trong df_result: KGN, VN, TVTU(DDN, VSN)

    # output
    - show các bước giải, nếu lựa chọn show
    - show kết quả cuối cùng: 
        + btoan có nghiêm duy nhất: show nghiêm tối ưu, giá trị tối ưu
        + btoan vô nghiệm: nếu optimize_direction là max thì giá trị tối ưu là -inf, ngược lại
        + btoan không giới nội: nếu optimize_direction là max thì giá trị tối ưu là inf, ngược lại
        + btoan vô số nghiệm: cho các biến không cơ sở xuất hiện ở hàm mục tiêu bằng 0, viết biến cơ sở theo điều kiện các biến không cơ sở
    """
    
    # tách type_result "TVTU" thành "DDN" và "VSN"
    arr_tvtu = df_result[-1] # ma trận từ vựng tối ưu
    tvtu = arr_tvtu[0, :-1] # từ vựng tối ưu không lấy bi
    if type_result == "TVTU":
        if np.all([np.count_nonzero(arr_tvtu[0,:-1]) <  (np.count_nonzero(row) - 1) for row in arr_tvtu[1:,:-1]]):
            type_result = "VSN"
        else:
            type_result = "DNN"


    # Xét các trường hợp 
    """ 
    - x_P_hat là ma trận chứa nghiệm của các biến x1, ..., xn của (P')
    - x_P_hat là ma trận chứa nghiệm của các biến x1, ..., xn của (P) aka kết quả x cuối cùng
    - z là giá trị tối ưu của bài toán
    """
    x_P_hat = None
    x_P = None
    z = None
    # 1. Duy nhất nghiệm
    if type_result == "DNN":
        arr_tvtu = df_result[-1]
        result = np.zeros(arr_tvtu.shape[1])
        result[-1] = arr_tvtu[0, -1]
        for col in range(df_result[-1].shape[1] - 1):
            ind = find_eye_list(arr_tvtu[:,col])
            if ind != None:
                result[col] = arr_tvtu[ind, -1]
            else:
                result[col] = 0
        n_const = arr_tvtu.shape[0] - 1
        x_P_hat = result[:(len(result) - n_const - 1)]
        x_P = rsby @ x_P_hat
        if optimize_direction == 'min':
            z = result[-1]
        else:
            z = -result[-1]
    
    # 2. Không giới nội
    elif type_result == "KGN":
        x_P = "Bai toan khong gioi noi"
        if optimize_direction == 'min':
            z = -np.inf
        else:
            z = np.inf

    # 3. Vô nghiệm
    elif type_result == "VN":
        x_P = "Bai toan vo nghiem"
        if optimize_direction == 'min':
            z = np.inf
        else:
            z = -np.inf

    elif type_result == "VSN":
        """
        Xét trên từ vựng tối ưu ci>=0. Nếu tồn tại ci = 0 --> hàm mục tiêu bị khuyết so với các ràng buộc --> VSN
        - Cho biến cơ sở ở hàm mục tiêu bằng 0
        - Viết biến cơ sở theo biến khuyết
        - Tìm điều kiện, hệ điều kiện cho biến khuyết
        + Chỉ có 1 biến khuyết: tìm khoảng giới hạn của biến khuyết (gộp chung với có 2 biến khuyết)
        + Có từ 2 biến khuyết trở lên: viết hệ điều kiện 
        """
        # x_P lúc này dùng để lưu điều kiện cho biến x1, x2,..., w1, w2, ... được lưu với cú pháp 0:">=", 1:"="
        # z để lưu hệ điều kiện cho biến dưới dạng ma trận dòng x1, x2, ..., wn; cột x1, x2, ..., wn
        x_P = tvtu
        tmp_arr_tvtu = arr_tvtu.copy()
        for i in range(len(x_P)):
            if x_P[i] != 0:
                x_P[i] = 1
                tmp_arr_tvtu[:, i] = 0
        z = np.zeros((tmp_arr_tvtu.shape[1] - 1, tmp_arr_tvtu.shape[1]))
        for i in range(z.shape[0]):
            j = find_eye_list(tmp_arr_tvtu[:,i])
            if j != None:
                z[i] = tmp_arr_tvtu[j]
                z[i,i] = 0

    
    return(x_P, z, df_result)

    



# 2 pha

In [248]:
def two_phase_algorithm(norm_arr, tab_arr):
    """ 
    # goal: sử dụng thuật toán 2 pha để giải quyết bài toán quy hoạch tuyến tính khi tồn tại ci<0 trên hàm mục tiêu và bi<0 trên ràng buộc đẳng thức, bất đẳng thức sau khi chuẩn hoá

    # input
    - norm_arr (np.array): ma trận từ vựng được tạo thành bởi hàm mục tiêu; các ràng buộc đẳng thức, bất đẳng thức; các ràng buộc về dấu của biến đã được chuẩn hoá
    - tab_arr (np.array): ma trận norm_arr sau khi được đưa về dạng bảng của phương pháp đơn hình

    # output
    - df_result (list): một danh sách lưu các từ vựng/ các bước giải của thuật toán 2 pha
    - type_result: phân loại kết quả của từ vựng cuối cùng trong df_result: KGN, VN, TVTU(DDN, VSN) 
    """

    # 1. Pha 1
    """ 
    1.1 Lập bài toán bổ trợ  
    - Thêm biến x0 vào ràng buộc đẳng thức, bất đẳng thức. Lập từ vựng xuất phát C = x0  
    - Đưa về dạng ma trận để có thể giải, lưu vào biến tab_C1 (np.array)
    """
    tab_C1 = tab_arr.copy()
    tab_C1 = np.insert(tab_C1, 0, np.ones(tab_C1.shape[0]), axis = 1)
    tab_C1[0, 1:] = 0

    """ 
    1.2 Xoay từ vựng 
    - Từ vựng đầu tiên
    + Chọn biến vào: x0
    + Chọn biến ra: biến ở dòng ứng với bi âm nhất, ưu tiên dòng có hệ số nhỏ nhất  
    - Từ vựng thứ 2 trở đi: áp dụng cách xoay đơn hình --> từ vựng cuối pha 1
    """
    # chọn biến vào là x_0
    j = 0
    # chọn biến ra, do ở thuật toán 2 pha, luôn tồn tại bi<0 nên luôn tồn tại biến ra i
    bi = tab_C1[1:, -1]
    i = np.where(bi == np.min(bi[bi<0]))[0][0] + 1
    # tạo df_result để lưu các từ vựng 
    df_result = list()
    # xoay từ vựng
    cur_arr = tab_C1
    df_result.append(cur_arr)
    in_out_ind = (j,i)
    while in_out_ind != 'TVTU':
        if in_out_ind != "KGN":
            j, i = in_out_ind
            new_arr = bland_rotate(cur_arr, i, j)
            df_result.append(new_arr)
        else:
            type_result = "KGN"
            break
        cur_arr = new_arr
        in_out_ind = select_input_output_index(cur_arr)
    

    """ 
    1.3 Kiểm tra từ vựng cuối pha 1 (từ vựng tối ưu của P1), xét hàm mục tiêu
    - Nếu hàm mục tiêu chỉ có biến x0 --> chuyển sang pha 2
    - Nếu hàm mục tiêu gồm biến x0 và những biến khác (hoặc không có x0) --> kết luận bài toán vô nghiệm --> dừng thuật toán. 
    """
    type_result = None
    # hàm mục tiêu của từ vựng cuối pha 1
    obj_func_last_C1 = df_result[-1][0,:-1]
    # Nếu hàm mục tiêu chỉ có biến x0 --> pha 2
    if obj_func_last_C1[0] == 1 and np.all(obj_func_last_C1[1:] == 0):
        # 2. Pha 2
        """ 
        2.1 Tạo từ vựng mới
        - Từ từ vựng tối ưu của pha 1 (P1) -> cho x_0 = 0, lấy các ràng buộc ở từ vựng này
        - Dùng hàm mục tiêu gốc (P) kết hợp với hàm mục tiêu (P1) -> hàm mục tiêu mới
        """
        # Xoá cột x_0
        temp_arr_last_C1 = df_result[-1].copy()
        temp_arr_last_C1 = np.delete(temp_arr_last_C1,0,1)
        # tìm số biến, số ràng buộc của bài toán gốc (P)
        n_const = temp_arr_last_C1.shape[0] - 1
        n_var = temp_arr_last_C1.shape[1]-temp_arr_last_C1.shape[0]
        # tạo một ma trận biểu diễn x1, x2,... từ hàm mục tiêu ban đầu
        obj_fun_z = norm_arr[0,:-1].reshape(1,-1)
        # tạo một ma trận biểu diễn x1, x2,... từ hàm mục tiêu theo x1, x2, ..., w1, w2, ... có được trong từ vựng cuối pha 1, với shape = temp_arr_last_C1.shape
        rsby_C1 = np.zeros((n_var, n_const + n_var + 1))

        for i in range(n_var):
            if find_eye_list(temp_arr_last_C1[:,i]) != None:
                j = find_eye_list(temp_arr_last_C1[:,i])
                rsby_C1[i] = temp_arr_last_C1[j]
                rsby_C1[i,i] = 0
            else:
                rsby_C1[i,i] = 1

        z = obj_fun_z@rsby_C1
        # kết hợp z với ràng buộc của từ vựng cuối pha 1 --> tạo thành từ vựng đầu pha 2 tab_C2
        tab_C2 = np.insert(z, 1, temp_arr_last_C1[1:], axis = 0)

        """ 
        2.2 Tiến hành xoay Bland trên từ vựng mới --> kết luận nghiệm
        """
        cur_arr = tab_C2
        df_result.append(cur_arr)
        # Xoay bland cho đến khi tìm được từ vựng tối ưu
        in_out_ind = select_input_output_index(cur_arr)
        while in_out_ind != 'TVTU':
            if in_out_ind != "KGN":
                j, i = in_out_ind
                new_arr = bland_rotate(cur_arr, i, j)
                df_result.append(new_arr)
            else:
                type_result = "KGN"
                break
            cur_arr = new_arr
            in_out_ind = select_input_output_index(cur_arr)

        type_result = select_input_output_index(df_result[-1])


    # Nếu hàm mục tiêu gồm biến x0 và những biến khác (hoặc không có x0) --> kết luận bài toán vô nghiệm --> dừng thuật toán. 
    else:
        type_result = "VN"

    return(df_result, type_result)





# Ví dụ

## 1. Giải bài toán bằng phương pháp Bland  


Lấy ví dụ câu 2.5d/63  

- A là ma trận chứa các ràng buộc về đẳng thức và bất đẳng thức  
z = min  5x1 - 10x2  
-2x1 +   x2  <= 1  
x1 - x2 >= -2  
3x1 + x2 <= 8
-2x1 + 3x2 >= -9  
4x1 + 3x2 >= 0

- B là ma trận chứa các ràng buộc về dấu của biến
x1 >= 0
x2 tự do

1. Bài toán có duy nhất nghiệm
5,-10,,min  
-2,1,<=,1  
1,-1,>=,-2  
3,1,<=,8  
-2,3,>=,-9  
4,3,>=,0  
>=,0

-10,57,9,24,,min
0.5,-5.5,-2.5,9,<=,0
0.5,-1.5,-0.5,1,<=,0
1,0,0,0,<=,1
>=,>=,>=,>=

2,-3,4,,min
,-2,-3,>=,-5
1,1,2,<=,4
1,2,3,<=,7
>=,>=,>=

-10,57,9,24,-100,,min
0.5,-5.5,-2.5,9,1,<=,1
0.5,-1.5,-0.5,1,1,<=,1
1,0,0,0,1,<=,1
0,0,0,0,1,<=,1
>=,>=,>=,>=,>=

2. Bài toán không giới nội
1,3,-1,,max
2,2,-1,<=,10
3,-2,1,<=,10
1,-3,1,<=,10
>=,>=,>=


In [249]:
A = pd.read_csv("input_const.csv", header=None)
B = pd.read_csv('input_vars.csv', header=None)

norm_arr, tab_arr, rsby, optimize_direction = processing_input(A,B)

df_result, type_result = bland_algoritm(tab_arr)
x_P, z, df_result = print_result(rsby, optimize_direction, df_result, type_result)
print(f"x = {x_P} \nz = {z} \n")
for i in df_result:
    print(i,'\n')

x = [0. 0. 0. 0. 1.] 
z = -100.0 

[[ -10.    57.     9.    24.  -100.     0.     0.     0.     0.     0. ]
 [  -0.5    5.5    2.5   -9.    -1.    -1.    -0.    -0.    -0.     1. ]
 [  -0.5    1.5    0.5   -1.    -1.    -0.    -1.    -0.    -0.     1. ]
 [  -1.    -0.    -0.    -0.    -1.    -0.    -0.    -1.    -0.     1. ]
 [  -0.    -0.    -0.    -0.    -1.    -0.    -0.    -0.    -1.     1. ]] 

[[  0.   57.    9.   24.  -90.    0.    0.   10.    0.  -10. ]
 [  0.    5.5   2.5  -9.   -0.5  -1.    0.    0.5   0.    0.5]
 [  0.    1.5   0.5  -1.   -0.5   0.   -1.    0.5   0.    0.5]
 [ -1.   -0.   -0.   -0.   -1.   -0.   -0.   -1.   -0.    1. ]
 [  0.    0.    0.    0.   -1.    0.    0.    0.   -1.    1. ]] 

[[ 0.000e+00 -9.330e+02 -4.410e+02  1.644e+03  0.000e+00  1.800e+02
   0.000e+00 -8.000e+01  0.000e+00 -1.000e+02]
 [ 0.000e+00  1.100e+01  5.000e+00 -1.800e+01 -1.000e+00 -2.000e+00
   0.000e+00  1.000e+00  0.000e+00  1.000e+00]
 [ 0.000e+00 -4.000e+00 -2.000e+00  8.000e+00  0.

Chú thích:  
Trong một số bài toán, nhìn vào từ vựng tối ưu cuối cùng, lẽ ra z = k. Nhưng z được trả về là z ~ k. Điều này là do vấn đề lưu trữ số thực của python.  

## 2. Giải bài toán bằng phương pháp 2 pha  


Lấy ví dụ câu 2.9a/64

- C là ma trận chứa các ràng buộc về đẳng thức và bất đẳng thức  
z = max -x1 -3x2 -x3  
2x1 - 5x2 + x3 <= 5  
2x1 - x2 + 2x3 <= 4  

- D là ma trận chứa các ràng buộc về dấu của biến  
x1 >= 0  
x2 >= 0  
x3 >= 0  

1. Bài toán duy nhất nghiệm
-1,-3,-1,,max
2,-5,1,<=,-5  
2,-1,2,<=,4 
>=,>=,>=

1,1,,min
-2,-1,<=,4
-2,4,<=,-8
-1,3,<=,-7
>=,>=


3,1,,max
-1,1,>=,1
-1,-1,<=,-3
2,1,<=,4
>=,>=

2. Bài toán vô nghiệm  
-3,-1,,min
1,-1,<=,-1
1,1,>=,3
2,1,<=,2
>=,>=

3. Bài toán không giới nội 
1,-1,,max
-2,1,<=,-1
-1,-2,<=,-2
>=,>=


In [250]:
C =  pd.read_csv("input_consts2.csv", header=None)
D = pd.read_csv('input_vars2.csv', header=None)

norm_arr, tab_arr, rsby, optimize_direction = processing_input(C, D)

df_result, type_result = two_phase_algorithm(norm_arr, tab_arr)
x_P, z, df_result = print_result(rsby, optimize_direction, df_result, type_result)
print(f"x = {x_P} \nz = {z} \n")
for i in df_result:
    print(i,'\n')


x = Bai toan vo nghiem 
z = inf 

[[ 1.  0.  0.  0.  0.  0.  0.]
 [ 1. -1.  1. -1. -0. -0. -1.]
 [ 1.  1.  1. -0. -1. -0. -3.]
 [ 1. -2. -1. -0. -0. -1.  2.]] 

[[ 0. -1. -1.  0.  1.  0.  3.]
 [ 0. -2.  0. -1.  1.  0.  2.]
 [-1. -1. -1.  0.  1.  0.  3.]
 [ 0. -3. -2.  0.  1. -1.  5.]] 

[[ 0.   0.  -1.   0.5  0.5  0.   2. ]
 [ 0.  -1.   0.  -0.5  0.5  0.   1. ]
 [-1.   0.  -1.   0.5  0.5  0.   2. ]
 [ 0.   0.  -2.   1.5 -0.5 -1.   2. ]] 

[[ 0.    0.    0.   -0.25  0.75  0.5   1.  ]
 [ 0.   -1.    0.   -0.5   0.5   0.    1.  ]
 [-1.    0.    0.   -0.25  0.75  0.5   1.  ]
 [ 0.    0.   -1.    0.75 -0.25 -0.5   1.  ]] 

[[ 0.   0.5  0.   0.   0.5  0.5  0.5]
 [ 0.  -2.   0.  -1.   1.   0.   2. ]
 [-1.   0.5  0.   0.   0.5  0.5  0.5]
 [ 0.  -1.5 -1.   0.   0.5 -0.5  2.5]] 

